In [1]:
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
# from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime


C:\workspace\ShipSupplies\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [39]:
# import tensorflow as tf

# # GPU 디바이스 목록 가져오기
# gpu_devices = tf.config.list_physical_devices('GPU')

# if len(gpu_devices) > 0:
#     print("사용 가능한 GPU가 있습니다.")
#     for device in gpu_devices:
#         print("GPU 디바이스 이름:", device.name)
# else:
#     print("사용 가능한 GPU가 없습니다.")


사용 가능한 GPU가 없습니다.


In [42]:
# !pip install bert-for-tf2
# !pip install tensorflow_hub

                                              0.0/100.6 kB ? eta -:--:--
     -----------------------------------     92.2/100.6 kB 5.5 MB/s eta 0:00:01
     -------------------------------------- 100.6/100.6 kB 2.9 MB/s eta 0:00:00


In [2]:
import tensorflow_hub as hub
from tensorflow.keras import layers
import bert

In [4]:
data = pd.read_csv('../data/raw_postpro.csv') # 오류가나면 추가해주세요 .encoding = 'cp949'
# 컬럼 삭제
df = data.drop(['청구서번호','No.',  '선박입고','완료 여부','리드타임_음수제거','청구량','견적','견적수량','견적화폐','견적단가','발주번호','발주','발주수량','발주금액','미입고 기간','리드타임','창고입고','창고입고수량','입고창고','창고출고','창고출고수량','출고선박','출고운반선','선박입고','선박입고수량','완료 여부'], axis=1)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20517 entries, 0 to 20516
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Subject      20517 non-null  object
 1   Machinery    20517 non-null  object
 2   Assembly     20517 non-null  object
 3   청구품목         20517 non-null  object
 4   Part No.1    20517 non-null  object
 5   Part No.2    20517 non-null  object
 6   key1         20517 non-null  object
 7   key2         20517 non-null  object
 8   발주처          20517 non-null  object
 9   D/T          20517 non-null  object
 10  Control No.  20517 non-null  object
 11  leadtime     20517 non-null  int64 
dtypes: int64(1), object(11)
memory usage: 1.9+ MB


In [6]:
df = df[['청구품목','발주처','Machinery', 'Assembly' , "key1",'key2']]
# 'Machinery', 'Assembly', '청구품목', 'Part No.1', 'Part No.2', 'key1', '발주처'

In [7]:
from sklearn import preprocessing
label_encoders = {}  # 각 열에 대한 LabelEncoder를 저장하기 위한 딕셔너리
columns_to_encode = ['key2']  # 인코딩을 수행할 열의 이름 리스트

for column in columns_to_encode:
    le = preprocessing.LabelEncoder()
    le.fit(df[column])
    label_encoders[column] = le # 딕셔너리에 저장
    df[column+"_encoded"] = le.transform(df[column]) # 새로운 encoding 된 컬럼 추가

In [8]:
df = df.drop(['key2'], axis=1)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20517 entries, 0 to 20516
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   청구품목          20517 non-null  object
 1   발주처           20517 non-null  object
 2   Machinery     20517 non-null  object
 3   Assembly      20517 non-null  object
 4   key1          20517 non-null  object
 5   key2_encoded  20517 non-null  int32 
dtypes: int32(1), object(5)
memory usage: 881.7+ KB


In [10]:
# text = df[[ '청구품목', '발주처','Machinery', 'Assembly' , "key1"]].apply(lambda row: ' '.join(row), axis=1)
df_text = df[['청구품목', '발주처', 'Machinery', 'Assembly', 'key1']].apply(lambda row: ' '.join(row), axis=1).to_frame(name='text')
df_text['key2'] = df['key2_encoded']

In [11]:
df_text[:20]

text  key2
0   SEAL-O-RING-STOR HAEIN Coporation_Cheonan NO.1...    18
1   OIL COOLER & LINES HAEIN Coporation_Cheonan NO...    18
2   WASHER HAEIN Coporation_Cheonan NO.2 GENERATOR...    18
3   BOLT-HIGH TEMP HAEIN Coporation_Cheonan NO.1 G...    18
4   SEAL HAEIN Coporation_Cheonan NO.1 GENERATOR E...    18
5   CORE CHARGES FOR CYLINDER PACK AS HAEIN Copora...    18
6   PUMP GP-F TFR-REMAN HAEIN Coporation_Cheonan N...    18
7   GEAR-WTR PUMP DR HAEIN Coporation_Cheonan NO.1...    18
8   GEAR-WTR PUMP DR HAEIN Coporation_Cheonan NO.3...    18
9   GEAR-WTR PUMP DR HAEIN Coporation_Cheonan NO.3...    18
10  GEAR-WTR PUMP DR HAEIN Coporation_Cheonan NO.1...    18
11  GEAR-WTR PUMP DR HAEIN Coporation_Cheonan NO.1...    18
12  GEAR-WTR PUMP DR HAEIN Coporation_Cheonan NO.3...    18
13  IMPELLER (주)정원펌프 NO.1 CONDENSER PUMP PUMP PARTS 3    45
14  SELF-ALIGNING ROLLER BEARING 대동베아링상사 BOW THRUS...     4
15  TAPERED ROLLER BEARING 대동베아링상사 BOW THRUSTER OV...     4
16  SELF-ALIGNING ROLLER BEARING 대동베아링상사 BOW THRUS...     4
17  TAPERED ROLLER BEARING 대동베아링상사 BOW THRUSTER OV...    25
18  EXHUAST GAS TEMP. INDICATOR, P5 PIRIOU NAVAL S...    25
19  EXHAUST GAS TEMPERATURE INDICATOR 0 TO 700℃ PI...    25

In [12]:
import re
def preprocess_text(sen):
    sentence = remove_tags(sen)
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)
    sentence = re.sub(r'\s+', ' ', sentence)
    return sentence

TAG_RE = re.compile(r'<[^>]+>')
def remove_tags(text):
    return TAG_RE.sub('', text)

en_text = []
sentences = list(df_text['text'])
for sen in sentences:
    en_text.append(preprocess_text(sen))

print(df_text.columns.values)

['text' 'key2']


In [23]:
# en_text

In [14]:
y = df_text.key2.values

In [15]:
y[:20]

array([18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 45,  4,  4,  4,
       25, 25, 25])

In [16]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [17]:
# !pip uninstall -y tensorflow

In [18]:
# !pip3 install -U "tensorflow==2.11.1"

In [21]:
import tensorflow as tf
print(tf.__version__)
print(tf.__file__)

2.11.1
C:\workspace\ShipSupplies\venv\lib\site-packages\tensorflow\__init__.py


In [26]:
def tokenize_text(text):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))
tokenized_text = [tokenize_text(en) for en in en_text]

In [39]:
# en_text[0],tokenized_text[0]

In [98]:
reviews_with_len = [[text, y[i], len(text)] # 토큰화된 text, key값, text 길이
                 for i, text in enumerate(tokenized_text)]
# reviews_with_len[:5]

In [99]:
# 섞고 key2 기준으로 정렬
import random
random.shuffle(reviews_with_len)
reviews_with_len.sort(key=lambda x: x[2])
reviews_with_len[:50]

[[[23365, 3796, 3796, 23365, 3796], 7, 5],
 [[13354, 5658, 4049, 3194, 20965, 13617], 0, 6],
 [[5009, 5658, 4049, 3194, 7956, 2132], 57, 6],
 [[5747, 5658, 4049, 3194, 7956, 3796], 10, 6],
 [[5009, 5658, 4049, 3194, 7956, 2132], 57, 6],
 [[16054, 5658, 4049, 3194, 20965, 14027], 8, 6],
 [[21605, 5658, 4049, 3194, 20965, 13103], 2, 6],
 [[10053, 5658, 4049, 3194, 7956, 3796], 8, 6],
 [[3500, 10764, 5658, 4049, 3194, 10764], 50, 6],
 [[16054, 5658, 4049, 3194, 20965, 14027], 8, 6],
 [[7744, 5658, 4049, 3194, 1055, 10216], 24, 6],
 [[7744, 5658, 4049, 3194, 1042, 10216], 24, 6],
 [[10764, 5658, 4049, 3194, 7956, 2132], 57, 6],
 [[9093, 7744, 9347, 29329, 8622, 2112], 48, 6],
 [[10353, 5658, 4049, 3194, 7956, 2132], 19, 6],
 [[5747, 5658, 4049, 3194, 7956, 3796], 10, 6],
 [[7744, 5658, 4049, 3194, 6718, 3847], 24, 6],
 [[10764, 5658, 4049, 3194, 7956, 2132], 57, 6],
 [[17490, 5658, 4049, 3194, 20965, 13103], 39, 6],
 [[7744, 5658, 4049, 3194, 6718, 3847], 24, 6],
 [[17490, 5658, 4049, 3194

In [100]:

sorted_text_labels = [(review_lab[0], review_lab[1]) for review_lab in reviews_with_len]
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_text_labels, output_types=(tf.int32, tf.int32))
BATCH_SIZE = 32
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))
next(iter(batched_dataset))

(<tf.Tensor: shape=(32, 7), dtype=int32, numpy=
 array([[23365,  3796,  3796, 23365,  3796,     0,     0],
        [13354,  5658,  4049,  3194, 20965, 13617,     0],
        [ 5009,  5658,  4049,  3194,  7956,  2132,     0],
        [ 5747,  5658,  4049,  3194,  7956,  3796,     0],
        [ 5009,  5658,  4049,  3194,  7956,  2132,     0],
        [16054,  5658,  4049,  3194, 20965, 14027,     0],
        [21605,  5658,  4049,  3194, 20965, 13103,     0],
        [10053,  5658,  4049,  3194,  7956,  3796,     0],
        [ 3500, 10764,  5658,  4049,  3194, 10764,     0],
        [16054,  5658,  4049,  3194, 20965, 14027,     0],
        [ 7744,  5658,  4049,  3194,  1055, 10216,     0],
        [ 7744,  5658,  4049,  3194,  1042, 10216,     0],
        [10764,  5658,  4049,  3194,  7956,  2132,     0],
        [ 9093,  7744,  9347, 29329,  8622,  2112,     0],
        [10353,  5658,  4049,  3194,  7956,  2132,     0],
        [ 5747,  5658,  4049,  3194,  7956,  3796,     0],
        

In [101]:
import math

TOTAL_BATCHES = math.ceil(len(sorted_text_labels) / BATCH_SIZE)
TEST_BATCHES = TOTAL_BATCHES // 10
batched_dataset.shuffle(TOTAL_BATCHES)
test_data = batched_dataset.take(TEST_BATCHES)
train_data = batched_dataset.skip(TEST_BATCHES)

In [125]:
test_data

<TakeDataset element_spec=(TensorSpec(shape=(None, None), dtype=tf.int32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [126]:
# for inputs, targets in train_data:
#     print(targets)


In [103]:
TOTAL_BATCHES, TEST_BATCHES

(642, 64)

In [104]:
class TEXT_MODEL(tf.keras.Model):
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn_filters=50,
                 dnn_units=512,
                 model_output_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)
        self.embedding = tf.keras.layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.cnn_layer1 = tf.keras.layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = tf.keras.layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = tf.keras.layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        self.lstm = tf.keras.layers.LSTM(128)
        
        self.pool = tf.keras.layers.GlobalMaxPool1D()
        self.dense_1 = tf.keras.layers.Dense(units=dnn_units, activation="relu")
        self.dropout = tf.keras.layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = tf.keras.layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = tf.keras.layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(l) 
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        l_3 = self.pool(l_3)
        
        concatenated = tf.concat([l_1, l_2, l_3], axis=-1) 
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
                # LSTM 레이어에 입력을 3D 형상으로 변환하여 전달
        lstm_output = self.lstm(tf.expand_dims(concatenated, axis=1))
        
        model_output = self.last_dense(concatenated)
        return model_output

In [105]:
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 100 #200
CNN_FILTERS = 50 #100
DNN_UNITS = 128 #256
OUTPUT_CLASSES = 61
DROPOUT_RATE = 0.1 # 0.2
NB_EPOCHS = 10
VOCAB_LENGTH
# 100 50 128 61 0.1 10 =>0.88

30522

In [106]:
text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)

In [107]:
if OUTPUT_CLASSES == 2:
    text_model.compile(loss="binary_crossentropy",
                       optimizer="adam",
                       metrics=["accuracy"])
else:
    text_model.compile(loss="sparse_categorical_crossentropy",
                       optimizer="adam",
                       metrics=["sparse_categorical_accuracy"])

text_model.fit(train_data, epochs=10)

Epoch 1/10


578/578 [==============================] - 14s 22ms/step - loss: 2.2108 - sparse_categorical_accuracy: 0.4939
Epoch 2/10
578/578 [==============================] - 13s 22ms/step - loss: 1.1821 - sparse_categorical_accuracy: 0.7262
Epoch 3/10
578/578 [==============================] - 13s 22ms/step - loss: 0.9153 - sparse_categorical_accuracy: 0.7787
Epoch 4/10
578/578 [==============================] - 13s 22ms/step - loss: 0.7972 - sparse_categorical_accuracy: 0.8022
Epoch 5/10
578/578 [==============================] - 13s 22ms/step - loss: 0.7225 - sparse_categorical_accuracy: 0.8183
Epoch 6/10
578/578 [==============================] - 13s 22ms/step - loss: 0.6731 - sparse_categorical_accuracy: 0.8276
Epoch 7/10
578/578 [==============================] - 13s 22ms/step - loss: 0.6329 - sparse_categorical_accuracy: 0.8327
Epoch 8/10
578/578 [==============================] - 13s 22ms/step - loss: 0.6044 - sparse_categorical_accuracy: 0.8390
Epoch 9/10
578/578 [=======================

In [108]:
results = text_model.evaluate(test_data)
print("10번 학습 : ", results)

64/64 [==============================] - 0s 3ms/step - loss: 0.3918 - sparse_categorical_accuracy: 0.8872
10번 학습 :  [0.39179104566574097, 0.88720703125]


In [109]:
text_model.fit(train_data, epochs=3)

Epoch 1/3
578/578 [==============================] - 13s 22ms/step - loss: 0.5383 - sparse_categorical_accuracy: 0.8508
Epoch 2/3
578/578 [==============================] - 13s 22ms/step - loss: 0.5201 - sparse_categorical_accuracy: 0.8535
Epoch 3/3
578/578 [==============================] - 13s 22ms/step - loss: 0.5113 - sparse_categorical_accuracy: 0.8538


In [110]:
results = text_model.evaluate(test_data)
print("13번 학습 : ", results)

64/64 [==============================] - 0s 4ms/step - loss: 0.3847 - sparse_categorical_accuracy: 0.9136
10번 학습 :  [0.38465532660484314, 0.91357421875]


In [111]:
text_model.fit(train_data, epochs=4)

Epoch 1/4
578/578 [==============================] - 13s 22ms/step - loss: 0.4946 - sparse_categorical_accuracy: 0.8572
Epoch 2/4
578/578 [==============================] - 13s 22ms/step - loss: 0.4870 - sparse_categorical_accuracy: 0.8583
Epoch 3/4
578/578 [==============================] - 13s 22ms/step - loss: 0.4751 - sparse_categorical_accuracy: 0.8587
Epoch 4/4
578/578 [==============================] - 13s 22ms/step - loss: 0.4663 - sparse_categorical_accuracy: 0.8597


In [112]:
results = text_model.evaluate(test_data)
print("17번 학습 : ", results)

64/64 [==============================] - 0s 3ms/step - loss: 0.4821 - sparse_categorical_accuracy: 0.8896
17번 학습 :  [0.4820931851863861, 0.8896484375]


## pred 는각각 61개 컬럼 라벨에서의 확률

In [113]:
pred = text_model.predict(test_data)

64/64 [==============================] - 0s 4ms/step


In [120]:
pred.shape

(2048, 61)

In [137]:
pred[0]

array([1.27434685e-09, 1.60510503e-08, 1.04998650e-08, 2.61683704e-06,
       5.16599528e-07, 6.67346214e-08, 1.97473260e-10, 9.99911308e-01,
       2.39404191e-11, 7.07977279e-06, 4.62410412e-07, 5.45044621e-10,
       5.86954096e-11, 7.52679516e-12, 1.86030314e-12, 3.93511909e-05,
       5.47097534e-10, 6.21539300e-08, 6.74654004e-08, 1.94295316e-10,
       8.10439424e-11, 1.99252326e-09, 3.57315753e-06, 4.66719854e-07,
       1.95864006e-10, 1.25312673e-12, 1.69085852e-05, 9.17054106e-12,
       6.15831439e-08, 1.09192766e-12, 4.24984492e-09, 4.50734561e-10,
       5.88129438e-08, 4.97628264e-07, 1.06747793e-08, 1.26026922e-09,
       4.15828955e-10, 1.41653609e-05, 9.24621979e-09, 8.35700842e-09,
       1.02246898e-10, 2.66049700e-11, 3.29004628e-08, 3.81697873e-08,
       4.16500896e-08, 3.94791932e-08, 6.06574204e-08, 2.25682550e-13,
       2.05820538e-09, 1.14616917e-07, 1.08290238e-10, 1.10450038e-07,
       1.37009522e-06, 7.30778353e-08, 1.05243515e-07, 3.69946491e-08,
      

# predicted_result 50개의 배치(배치 사이즈 : 32) 데이터 마다 예측 라벨링을 리스트에 저장 

In [191]:
predicted_result = []
for j in range(64):
    temp=[]
    for i in range(32) :   
        predicted_class = tf.argmax(pred[i+ j*32]).numpy() ## 가장 높은 확률의 라벨링 데이터를 구함
        temp.append(predicted_class)
    predicted_result.append(temp)
        # print(predicted_class)

In [192]:
for lis in predicted_result:
    print(lis)

[7, 0, 25, 10, 25, 8, 2, 8, 50, 8, 24, 24, 57, 48, 19, 10, 24, 57, 39, 24, 39, 25, 4, 50, 4, 48, 23, 50, 4, 17, 24, 44]
[23, 46, 44, 44, 39, 39, 57, 7, 46, 45, 39, 44, 50, 24, 8, 19, 4, 58, 40, 23, 4, 8, 23, 46, 24, 8, 7, 54, 8, 17, 39, 7]
[58, 8, 44, 19, 39, 24, 50, 57, 8, 39, 40, 17, 39, 58, 54, 57, 57, 17, 58, 10, 7, 57, 41, 57, 0, 50, 17, 40, 40, 45, 17, 40]
[8, 8, 24, 24, 24, 24, 17, 23, 4, 44, 17, 8, 23, 46, 17, 26, 7, 19, 23, 39, 8, 17, 39, 5, 39, 4, 44, 16, 24, 30, 44, 23]
[43, 4, 7, 0, 44, 24, 4, 17, 4, 7, 4, 23, 17, 24, 46, 27, 24, 39, 46, 45, 40, 24, 57, 55, 40, 46, 25, 10, 58, 42, 24, 50]
[4, 48, 57, 57, 57, 24, 9, 24, 18, 51, 10, 46, 7, 40, 50, 24, 57, 8, 57, 37, 55, 40, 46, 49, 57, 56, 10, 26, 24, 40, 54, 40]
[24, 19, 23, 58, 40, 58, 24, 58, 39, 57, 24, 24, 24, 42, 24, 51, 25, 25, 25, 45, 4, 54, 24, 24, 24, 58, 54, 42, 54, 7, 21, 4]
[58, 24, 24, 44, 40, 57, 45, 24, 24, 20, 57, 40, 46, 57, 24, 38, 49, 8, 54, 4, 41, 58, 0, 57, 40, 40, 8, 24, 45, 19, 42, 4]
[41, 22, 58, 58, 

# true_result : 테스트 데이터에서의 실제 라벨링

In [193]:
true_result =[]
for inputs, targets in test_data.take(64):
    # 첫 번째 데이터 샘플에 대한 입력(inputs)과 라벨(targets)을 확인

    # Convert EagerTensor to numpy array
    targets_numpy = np.array(targets)

    # Convert numpy array to list
    targets_list = targets_numpy.tolist()
    true_result.append(targets_list)
    # print("Targets:", targets_list)


In [194]:
for lis in true_result:
    print(lis)

[7, 0, 57, 10, 57, 8, 2, 8, 50, 8, 24, 24, 57, 48, 19, 10, 24, 57, 39, 24, 39, 24, 4, 50, 4, 48, 23, 50, 4, 17, 24, 44]
[23, 46, 44, 44, 39, 39, 57, 7, 46, 45, 39, 44, 50, 24, 8, 19, 4, 58, 24, 23, 4, 8, 23, 46, 24, 8, 7, 54, 8, 17, 39, 7]
[24, 8, 44, 44, 39, 18, 50, 57, 8, 39, 40, 17, 39, 58, 54, 57, 57, 17, 58, 10, 7, 57, 41, 57, 0, 50, 17, 40, 40, 45, 17, 24]
[8, 8, 40, 24, 24, 24, 17, 23, 4, 44, 52, 8, 23, 46, 17, 26, 7, 19, 23, 39, 8, 17, 39, 5, 39, 4, 44, 17, 24, 24, 44, 23]
[43, 4, 7, 0, 44, 24, 24, 17, 4, 7, 4, 23, 17, 24, 46, 27, 24, 39, 46, 18, 40, 24, 57, 55, 40, 46, 25, 10, 58, 42, 24, 50]
[4, 48, 57, 57, 57, 24, 26, 24, 2, 51, 10, 46, 7, 40, 50, 58, 57, 8, 57, 37, 55, 40, 46, 49, 57, 12, 10, 26, 24, 40, 54, 24]
[24, 19, 23, 58, 40, 58, 24, 58, 39, 57, 24, 24, 24, 42, 24, 51, 25, 25, 25, 45, 4, 54, 24, 24, 24, 58, 54, 42, 54, 7, 21, 4]
[58, 24, 24, 44, 40, 57, 18, 24, 24, 20, 57, 40, 46, 57, 24, 9, 49, 8, 54, 4, 41, 58, 0, 57, 40, 40, 8, 24, 45, 19, 42, 4]
[41, 22, 58, 58, 

In [195]:
from sklearn.metrics import f1_score,classification_report

In [196]:
# Calculate F1 score
f1 = f1_score([item for sublist in true_result for item in sublist],
              [item for sublist in predicted_result for item in sublist], average='macro')
print("F1 score:", f1)

F1 score: 0.7718434509223613


In [197]:
print(classification_report([item for sublist in predicted_result for item in sublist],
                            [item for sublist in true_result for item in sublist]))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       0.00      0.00      0.00         1
           2       0.67      0.60      0.63        10
           3       0.25      0.50      0.33         2
           4       1.00      0.96      0.98       183
           5       1.00      0.67      0.80         3
           7       0.88      1.00      0.94        22
           8       0.99      1.00      0.99        98
           9       0.60      0.75      0.67         4
          10       1.00      1.00      1.00        33
          12       0.76      1.00      0.86        16
          13       1.00      1.00      1.00         3
          14       0.00      0.00      0.00         0
          15       0.00      0.00      0.00         1
          16       1.00      0.38      0.55        16
          17       0.50      0.84      0.63        19
          18       0.69      0.69      0.69        13
          19       0.86    

C:\workspace\ShipSupplies\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\workspace\ShipSupplies\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\workspace\ShipSupplies\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\workspace\ShipSupplies\venv\lib\site-packa